In [ ]:
!pip -q install langchain openai tiktoken huggingface_hub

In [ ]:
!pip -q install transformers datasets

In [ ]:
from langchain.evaluation.loading import load_dataset
dataset = load_dataset("question-answering-state-of-the-union")

In [9]:
dataset[0]

{'question': 'What is the purpose of the NATO Alliance?',
 'answer': 'The purpose of the NATO Alliance is to secure peace and stability in Europe after World War 2.'}

In [58]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.chains import QAGenerationChain

In [11]:
with open('/content/vim_play.txt') as vim:
  data = vim.read()

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

vim_split = RecursiveCharacterTextSplitter(chunk_size=100,
                                           chunk_overlap=25,
                                           length_function=len)

In [14]:
text_docs = vim_split.split_text(data)

In [45]:
import os
os.environ['OPENAI_API_KEY']=''

In [62]:
chain = QAGenerationChain.from_llm(ChatOpenAI(temperature = 0))

In [40]:
qa = chain.run(text_docs[0:5])

In [61]:
text_docs[0]

'Vim Play book inside the SpaceVim flavor'

In [63]:
chain.run(text_docs[0])

[{'question': 'What is the Vim Play book?',
  'answer': 'The Vim Play book is a feature inside the SpaceVim flavor.'}]

In [64]:
import time
qa_dataset = []

for l in text_docs[:5]:
  time.sleep(60)
  data_gen = chain.run(l)
  qa_dataset.append(data_gen)

In [69]:
qa_dataset = [l[0] for l in qa_dataset]

In [22]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings

In [ ]:
!pip install faiss-cpu

In [23]:
from langchain.vectorstores import FAISS

In [24]:
embed_func = OpenAIEmbeddings()

In [26]:
faissDB = FAISS.from_texts(texts=text_docs[:2],
                           embedding=embed_func)

In [35]:
eval_chain = RetrievalQA.from_chain_type(llm=OpenAI(), 
                                         chain_type="stuff",
                                         retriever=faissDB.as_retriever(), 
                                         input_key="question")

In [30]:
len(qa)

1

In [71]:
predictions = eval_chain.apply(qa_dataset)

In [72]:
predictions[0]

{'question': 'What is the Vim Play book?',
 'answer': 'The Vim Play book is a feature inside the SpaceVim flavor.',
 'result': ' The Vim Play book is a document inside the SpaceVim flavor that lists the various activities.'}

In [31]:
from langchain.evaluation.qa import QAEvalChain

In [37]:
llm = OpenAI(temperature=0)
qaeval_chain = QAEvalChain.from_llm(llm)

In [73]:
graded_outputs = qaeval_chain.evaluate(qa_dataset, 
                                     predictions, 
                                     question_key="question", 
                                     prediction_key="result")

In [74]:
graded_outputs

[{'text': ' CORRECT'},
 {'text': ' CORRECT'},
 {'text': ' INCORRECT'},
 {'text': ' CORRECT'},
 {'text': ' INCORRECT'}]